In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests

In [3]:
def frist_name(browser, num):
    try:
        name = browser.find_element(By.XPATH, f'//*[@id="content"]/div[3]/ul/li[{num}]/div/div[1]/strong/a[1]/strong').text
    except:
        name = 'Nan'

    return name


def frist_rating(browser, num):
    try:
        rating = browser.find_element(By.CSS_SELECTOR, f'#content > div.list_wrap > ul > li:nth-child({num}) > div.info_area > div.related.v2 > div > span:nth-child(2) > div > a > span.u_likeit_text.num._count').text
    except:
        rating = browser.find_element(By.CSS_SELECTOR, f'#content > div.list_wrap > ul > li:nth-child({num}) > div.info_area > div.related.v2 > div > span:nth-child(2) > div > a > span.u_likeit_text._count').text     

    return rating


def frist_url(browser, num):
    try:
        url = browser.find_element(By.XPATH, f'//*[@id="content"]/div[3]/ul/li[{num}]/div/div[1]/strong/a[1]').get_attribute('href')
    except:
        url = 'Nan'
    
    return url


def page_check(browser):
    if browser.find_element(By.XPATH, '//*[@id="paginate"]/span[2]/a').get_attribute('href')[-1] == '#':
        return True

In [4]:
r = open('pill_list.txt', 'r')
pill_list = [i.rstrip() for i in r.readlines()]

In [5]:
pill_list.__len__()

2031

In [6]:
df_url = pd.DataFrame(columns=['이름', 'rating', 'url'])
df_url_idx = 0

In [7]:
url = f'https://terms.naver.com/medicineSearch.naver?mode=nameSearch&query=' #{}&page=1

In [8]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [9]:
for pill in pill_list[:]:
    page = 1
    while True:
        browser.get(url + pill + '&page=' + str(page))
        time.sleep(1.5) # 인터넷 상태에 따라서 조정하여 사용. 너무 짧을 경우 rating이 수집되지 않는 경우가 있음.
        # browser.implicitly_wait(1.5)
        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        for i in range(1, 10+1):
            try:
                pill_rating = frist_rating(browser, i) 
            except:
                break

            pill_name = frist_name(browser, i)
            if pill_name in df_url['이름']: continue

            pill_url = frist_url(browser, i)


            df_url.loc[df_url_idx] = [pill_name, pill_rating, pill_url]
            # print([pill_name, pill_rating, pill_url])
            df_url_idx += 1
            
        page += 1

        if page_check: break


In [23]:
df_url['rating'] = df_url['rating'].apply(lambda x: 0 if x == '공감' else x)
df_url.to_csv('url_df.csv', encoding='utf-8-sig', index=False)
df_url['rating'] = df_url['rating'].astype(int)

In [24]:
target_url = df_url[(df_url['rating']>=5)].reset_index(drop=True)
target_url

,이름,rating,url
0,뉴본정50단위(칼리디노게나제) [Newborn Tab. 50unit],27,https://terms.naver.com/entry.naver?docId=2135...
1,크레인정50단위(칼리디노게나제) [Krein Tab.],29,https://terms.naver.com/entry.naver?docId=2141...
2,카레인정50단위(칼리디노게나제) [Kalein Tab. 50IU],14,https://terms.naver.com/entry.naver?docId=2164...
3,튜비스투정150/300mg [TUBES2 Tablet 150/300mg],10,https://terms.naver.com/entry.naver?docId=2164...
4,튜비스정 [TUBES Tablet],8,https://terms.naver.com/entry.naver?docId=2128...
...,...,...,...
562,콜락연질캡슐 [Collag soft cap.],12,https://terms.naver.com/entry.naver?docId=2131...
563,젠빅연질캡슐 [Genbig Soft Cap.],13,https://terms.naver.com/entry.naver?docId=2155...
564,쎄투연질캡슐 [Cettu Soft Capsule],16,https://terms.naver.com/entry.naver?docId=2134...
565,레비아정(날트렉손염산염) [Revia Tab. (Naltrexone Hcl)],13,https://terms.naver.com/entry.naver?docId=2141...


In [25]:
final_df = pd.DataFrame(columns=["이름", "성분정보", "효능효과", "용법용량", "저장방법", "사용기간", "사용상주의사항", 'url'])
final_df_idx = 0

In [26]:
for i in range(len(target_url)):
    res = requests.get(target_url.iloc[i,2])
    res.raise_for_status()                
    soup = BeautifulSoup(res.text, "lxml")
    temp = soup.find_all('p', {'class': 'txt'})
    temp = list(temp)

    try:
        final_df.loc[final_df_idx] = [target_url.iloc[i,0], 
                                    temp[0], 
                                    temp[1],
                                    temp[2],
                                    temp[3],
                                    temp[4],
                                    temp[5],
                                    target_url.iloc[i,2]]
        final_df_idx += 1
    except:
        final_df.loc[final_df_idx] = [target_url.iloc[i,0], 
                                    "Nan",
                                    temp[0], 
                                    temp[1],
                                    temp[2],
                                    temp[3],
                                    temp[4],
                                    # temp[5],
                                    target_url.iloc[i,2]]
        final_df_idx += 1
        print(target_url.iloc[i,2])

https://terms.naver.com/entry.naver?docId=2128400&cid=51000&categoryId=51000
https://terms.naver.com/entry.naver?docId=2128400&cid=51000&categoryId=51000


In [ ]:
def pre_word(x):
    return str(x).replace('<p class="txt">',"").replace('<b>',"").replace('</b>',"").replace("</p>","").rstrip().replace("<br/>","\n").replace("&gt;",">").replace("&lt;","<")

for c in final_df.columns.tolist():
    final_df[c] = final_df[c].apply(lambda x: pre_word(x).lstrip().rstrip())

In [ ]:
final_df.to_csv('final_df.csv', index=False, encoding='utf-8-sig')